In [2]:
import pandas as pd

import fnmatch
import glob
import dropbox as db

# Import Files from Dropbox Connection

In [21]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import dbx_token
dbx = db.Dropbox(dbx_token)

path = '/Users/matthiashugli/Dropbox/bucket/euro2020/*.csv'
# suisse_path = '/Users/matthiashugli/Dropbox/bucket/euro2020/raw-suisse-shots-against.csv'

df = pd.DataFrame()
for filename in glob.glob(path):
    #print(filename)
    file = pd.read_csv(filename, header=0, delimiter=',', low_memory=False)
    file.insert(0, 'filename', filename)
    df = df.append(file)

# suisse_df = pd.DataFrame()
# for filename in glob.glob(sommer_path):
#     #print(filename)
#     file = pd.read_csv(filename, header=1, delimiter=',', low_memory=False)
#     file.insert(0, 'filename', filename)
#     suisse_df = suisse_df.append(file)

ln: config.py: File exists


In [22]:
df[['source', 'file']] = df.filename.apply(lambda x: pd.Series(str(x).split('raw-')))
df[['goalkeeper', 'split6']] = df.file.apply(lambda x: pd.Series(str(x).split('.csv')))
df = df.drop(columns=['source', 'file', 'split6'])
print(df.head(5))

                                            filename        Date  Day  \
0  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-15  Tue   
1  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-19  Sat   
2  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-23  Wed   
3  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-28  Mon   
0  /Users/matthiashugli/Dropbox/bucket/euro2020/r...  2021-06-14  Mon   

         Round    Venue         Result              Squad        Opponent  \
0  Group stage  Neutral          W 1–0          fr France      de Germany   
1  Group stage  Neutral          D 1–1          fr France      hu Hungary   
2  Group stage  Neutral          D 2–2          fr France     pt Portugal   
3  Round of 16  Neutral  D 3 (4)–3 (5)          fr France  ch Switzerland   
0  Group stage  Neutral          W 2–0  cz Czech Republic    sct Scotland   

  Start Pos  ...  Att.2  Launch%.1  AvgLen.1  Opp  Stp  Stp%  #OPA  AvgDist  \
0    Y*  GK  ...   

In [23]:
xSaves = df.loc[:, ('goalkeeper', 'PSxG', 'Saves', 'SoTA', 'GA')]
xSaves['Saves%'] = xSaves['Saves'] / xSaves['SoTA']
xSaves['xS'] = xSaves['SoTA'] - xSaves['PSxG']
xSaves['xS%'] = xSaves['xS'] / xSaves['SoTA']

In [24]:
gk_xSaves = xSaves.groupby('goalkeeper').agg({'Saves': 'sum', 'xS': 'sum', 'SoTA': 'sum', 'PSxG': 'sum'})
gk_xSaves['xS%'] = gk_xSaves.xS / gk_xSaves.SoTA
gk_xSaves['Saves%'] = gk_xSaves.Saves / gk_xSaves.SoTA
gk_xSaves['shot_performance'] = gk_xSaves['Saves%'] - gk_xSaves['xS%']
gk_xSaves['PSxG/SoTA'] = gk_xSaves['PSxG'] / gk_xSaves.SoTA
print(gk_xSaves)

            Saves    xS  SoTA  PSxG       xS%    Saves%  shot_performance  \
goalkeeper                                                                  
dommarumma      9   6.4    12   5.6  0.533333  0.750000          0.216667   
loris           7   5.9    11   5.1  0.536364  0.636364          0.100000   
neuer           4   5.2    10   4.8  0.520000  0.400000         -0.120000   
olsen          11  11.0    15   4.0  0.733333  0.733333          0.000000   
patricio       10  10.0    15   5.0  0.666667  0.666667          0.000000   
pickford       14  14.4    17   2.6  0.847059  0.823529         -0.023529   
schmeichel     16  13.8    21   7.2  0.657143  0.761905          0.104762   
sommer         22  21.1    30   8.9  0.703333  0.733333          0.030000   
vaclik         11  10.6    16   5.4  0.662500  0.687500          0.025000   

            PSxG/SoTA  
goalkeeper             
dommarumma   0.466667  
loris        0.463636  
neuer        0.480000  
olsen        0.266667  
patricio